In [ ]:
import pandas as pd
import numpy as np

from fuzzywuzzy import process
from ast import literal_eval

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d marlesson/myanimelist-dataset-animes-profiles-reviews
!kaggle datasets download -d vishalmane10/anime-dataset-2022

 89% 193M/217M [00:01<00:00, 116MB/s]
100% 217M/217M [00:01<00:00, 119MB/s]
  0% 0.00/4.74M [00:00<?, ?B/s]
100% 4.74M/4.74M [00:00<00:00, 86.0MB/s]


In [ ]:
!unzip -q myanimelist-dataset-animes-profiles-reviews.zip -d myanimelist-dataset
!unzip -q anime-dataset-2022.zip -d anime-dataset

In [ ]:
csv_file_path1 = '/content/myanimelist-dataset/animes.csv'
anime = pd.read_csv(csv_file_path1)
csv_file_path2 = '/content/myanimelist-dataset/profiles.csv'
users = pd.read_csv(csv_file_path2)
csv_file_path3 = '/content/myanimelist-dataset/reviews.csv'
ratings = pd.read_csv(csv_file_path3)
csv_file_path4 = '/content/anime-dataset/Anime.csv'
authors = pd.read_csv(csv_file_path4)

In [ ]:
authors

,Rank,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Release_year,End_year,Description,Content_Warning,Related_Mange,Related_anime,Voice_actors,staff
0,1,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Kimetsu no Yaiba: Yuukaku-hen,TV,NaN,ufotable,Fall,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.60,2021.0,NaN,'Tanjiro and his friends accompany the Hashira...,Explicit Violence,Demon Slayer: Kimetsu no Yaiba,"Demon Slayer: Kimetsu no Yaiba, Demon Slayer: ...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk...","Koyoharu Gotouge : Original Creator, Haruo Sot..."
1,2,Fruits Basket the Final Season,Fruits Basket the Final,TV,13.0,TMS Entertainment,Spring,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60,2021.0,NaN,'The final arc of Fruits Basket.',"Emotional Abuse,, Mature Themes,, Physical Abu...","Fruits Basket, Fruits Basket Another","Fruits Basket 1st Season, Fruits Basket 2nd Se...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum...","Natsuki Takaya : Original Creator, Yoshihide I..."
2,3,Mo Dao Zu Shi 3,The Founder of Diabolism 3,Web,12.0,B.C MAY PICTURES,NaN,"Fantasy, Ancient China, Chinese Animation, Cul...",4.58,2021.0,NaN,'The third season of Mo Dao Zu Shi.',NaN,Grandmaster of Demonic Cultivation: Mo Dao Zu ...,"Mo Dao Zu Shi 2, Mo Dao Zu Shi Q","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang...","Mo Xiang Tong Xiu : Original Creator, Xiong Ke..."
3,4,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Full Metal Alchemist,TV,64.0,Bones,Spring,"Action, Adventure, Drama, Fantasy, Mystery, Sh...",4.58,2009.0,2010.0,"""The foundation of alchemy is based on the law...","Animal Abuse,, Mature Themes,, Violence,, Dome...","Fullmetal Alchemist, Fullmetal Alchemist (Ligh...","Fullmetal Alchemist: Brotherhood Specials, Ful...","Alphonse Elric : Rie Kugimiya, Edward Elric : ...","Hiromu Arakawa : Original Creator, Yasuhiro Ir..."
4,5,Attack on Titan 3rd Season: Part II,Shingeki no Kyojin Season 3: Part II,TV,10.0,WIT Studio,Spring,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",4.57,2019.0,NaN,'The battle to retake Wall Maria begins now! W...,"Cannibalism,, Explicit Violence","Attack on Titan, Attack on Titan: End of the W...","Attack on Titan, Attack on Titan 2nd Season, A...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu...","Hajime Isayama : Original Creator, Tetsurou Ar..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18490,18491,Qin Shi Mingyue: Canghai Hengliu Xiaomeng Spec...,NaN,Web,2.0,Sparkly Key Animation Studio,NaN,"Action, Ancient China, Chinese Animation, Hist...",NaN,2020.0,NaN,Special episodes of Qin Shi Mingyue: Canghai H...,NaN,NaN,Qin Shi Mingyue: Canghai Hengliu,NaN,NaN
18491,18492,Yi Tang Juchang: Sanguo Yanyi,NaN,TV,108.0,NaN,NaN,Chinese Animation,NaN,2010.0,NaN,No synopsis yet - check back soon!,NaN,NaN,NaN,NaN,NaN
18492,18493,Fenghuang Ji Xiang Yu Qingming Shanghe Tu,NaN,TV,13.0,NaN,NaN,"Chinese Animation, Family Friendly, Short Epis...",NaN,2020.0,NaN,No synopsis yet - check back soon!,NaN,NaN,NaN,NaN,NaN
18493,18494,Chengshi Jiyi Wo Men de Jieri,NaN,TV,NaN,NaN,NaN,"Chinese Animation, Family Friendly, Short Epis...",NaN,2020.0,NaN,No synopsis yet - check back soon!,NaN,NaN,NaN,NaN,NaN


In [ ]:
authors.head(1)['Voice_actors'].values

array(['Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuko Kamado : Akari Kitou, Tanjirou Kamado : Natsuki Hanae, Zenitsu Agatsuma : Hiro Shimono, Daki : Miyuki Sawashiro, Tengen Uzui : Katsuyuki Konishi, Akaza : Akira Ishida, Amane Ubuyashiki, Koyoharu Gotouge\nOriginal Creator, Haruo Sotozaki\nDirector, Akira Matsushima\nCharacter Design, Aimer\nSong Performance'],
      dtype=object)

In [ ]:
authors = authors[['Name', 'Studio', 'Tags', 'Description', 'staff', 'Voice_actors']]

In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,ufotable,"Action, Adventure, Fantasy, Shounen, Demons, H...",'Tanjiro and his friends accompany the Hashira...,"Koyoharu Gotouge : Original Creator, Haruo Sot...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk..."
1,Fruits Basket the Final Season,TMS Entertainment,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",'The final arc of Fruits Basket.',"Natsuki Takaya : Original Creator, Yoshihide I...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum..."
2,Mo Dao Zu Shi 3,B.C MAY PICTURES,"Fantasy, Ancient China, Chinese Animation, Cul...",'The third season of Mo Dao Zu Shi.',"Mo Xiang Tong Xiu : Original Creator, Xiong Ke...","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang..."
3,Fullmetal Alchemist: Brotherhood,Bones,"Action, Adventure, Drama, Fantasy, Mystery, Sh...","""The foundation of alchemy is based on the law...","Hiromu Arakawa : Original Creator, Yasuhiro Ir...","Alphonse Elric : Rie Kugimiya, Edward Elric : ..."
4,Attack on Titan 3rd Season: Part II,WIT Studio,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",'The battle to retake Wall Maria begins now! W...,"Hajime Isayama : Original Creator, Tetsurou Ar...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu..."
...,...,...,...,...,...,...
18490,Qin Shi Mingyue: Canghai Hengliu Xiaomeng Spec...,Sparkly Key Animation Studio,"Action, Ancient China, Chinese Animation, Hist...",Special episodes of Qin Shi Mingyue: Canghai H...,NaN,NaN
18491,Yi Tang Juchang: Sanguo Yanyi,NaN,Chinese Animation,No synopsis yet - check back soon!,NaN,NaN
18492,Fenghuang Ji Xiang Yu Qingming Shanghe Tu,NaN,"Chinese Animation, Family Friendly, Short Epis...",No synopsis yet - check back soon!,NaN,NaN
18493,Chengshi Jiyi Wo Men de Jieri,NaN,"Chinese Animation, Family Friendly, Short Epis...",No synopsis yet - check back soon!,NaN,NaN


In [ ]:
authors.dropna(inplace = True)

<ipython-input-12-0c983cabff9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors.dropna(inplace = True)


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,ufotable,"Action, Adventure, Fantasy, Shounen, Demons, H...",'Tanjiro and his friends accompany the Hashira...,"Koyoharu Gotouge : Original Creator, Haruo Sot...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk..."
1,Fruits Basket the Final Season,TMS Entertainment,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",'The final arc of Fruits Basket.',"Natsuki Takaya : Original Creator, Yoshihide I...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum..."
2,Mo Dao Zu Shi 3,B.C MAY PICTURES,"Fantasy, Ancient China, Chinese Animation, Cul...",'The third season of Mo Dao Zu Shi.',"Mo Xiang Tong Xiu : Original Creator, Xiong Ke...","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang..."
3,Fullmetal Alchemist: Brotherhood,Bones,"Action, Adventure, Drama, Fantasy, Mystery, Sh...","""The foundation of alchemy is based on the law...","Hiromu Arakawa : Original Creator, Yasuhiro Ir...","Alphonse Elric : Rie Kugimiya, Edward Elric : ..."
4,Attack on Titan 3rd Season: Part II,WIT Studio,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",'The battle to retake Wall Maria begins now! W...,"Hajime Isayama : Original Creator, Tetsurou Ar...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,TYO Animations,"Drama, Sports, Basketball, School Club",No synopsis yet - check back soon!,"Toshinao Aoki : Original Character Design, Kon...","Narrator : Haruka Ayase, Toshinao Aoki\nOrigin..."
18446,Ometeotl Hero,Speed,"Sci Fi, Anime Tamago, Shorts, Superpowers, Ori...","In the year 22XX, due to the drastic decrease ...",Jun Awazu : Director,"Yuki, Jun Awazu\nDirector"
18482,Boris: Free,foodunited.,Abstract,The official music video for Free from the alb...,"Ryuuta Murayama : Director, Atsuo : Director, ...","Ryuuta Murayama\nDirector, Atsuo\nDirector, Sh..."
18487,Make My Day,Studio 5,"Horror, Monsters, Original Work","On a cold planet of ice and snow, mysterious c...","Yasuo Ootagaki : Original Creator, Makoto Kane...","Yasuo Ootagaki\nOriginal Creator, Makoto Kanem..."


In [ ]:
authors.isnull().sum()

Name            0
Studio          0
Tags            0
Description     0
staff           0
Voice_actors    0
dtype: int64

In [ ]:
authors.duplicated().sum()

4

In [ ]:
authors.drop_duplicates(inplace=True)

<ipython-input-16-ed3f23405da7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors.drop_duplicates(inplace=True)


In [ ]:
authors.duplicated().sum()

0

In [ ]:
authors['Tags'] = authors['Tags'].str.split(', ')
authors['Tags'] = authors['Tags'].apply(lambda x: [genre.strip() for genre in x])

<ipython-input-18-7f5bf35f2e38>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Tags'] = authors['Tags'].str.split(', ')
<ipython-input-18-7f5bf35f2e38>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Tags'] = authors['Tags'].apply(lambda x: [genre.strip() for genre in x])


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,ufotable,"[Action, Adventure, Fantasy, Shounen, Demons, ...",'Tanjiro and his friends accompany the Hashira...,"Koyoharu Gotouge : Original Creator, Haruo Sot...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk..."
1,Fruits Basket the Final Season,TMS Entertainment,"[Drama, Fantasy, Romance, Shoujo, Animal Trans...",'The final arc of Fruits Basket.',"Natsuki Takaya : Original Creator, Yoshihide I...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum..."
2,Mo Dao Zu Shi 3,B.C MAY PICTURES,"[Fantasy, Ancient China, Chinese Animation, Cu...",'The third season of Mo Dao Zu Shi.',"Mo Xiang Tong Xiu : Original Creator, Xiong Ke...","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang..."
3,Fullmetal Alchemist: Brotherhood,Bones,"[Action, Adventure, Drama, Fantasy, Mystery, S...","""The foundation of alchemy is based on the law...","Hiromu Arakawa : Original Creator, Yasuhiro Ir...","Alphonse Elric : Rie Kugimiya, Edward Elric : ..."
4,Attack on Titan 3rd Season: Part II,WIT Studio,"[Action, Fantasy, Horror, Shounen, Dark Fantas...",'The battle to retake Wall Maria begins now! W...,"Hajime Isayama : Original Creator, Tetsurou Ar...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,TYO Animations,"[Drama, Sports, Basketball, School Club]",No synopsis yet - check back soon!,"Toshinao Aoki : Original Character Design, Kon...","Narrator : Haruka Ayase, Toshinao Aoki\nOrigin..."
18446,Ometeotl Hero,Speed,"[Sci Fi, Anime Tamago, Shorts, Superpowers, Or...","In the year 22XX, due to the drastic decrease ...",Jun Awazu : Director,"Yuki, Jun Awazu\nDirector"
18482,Boris: Free,foodunited.,[Abstract],The official music video for Free from the alb...,"Ryuuta Murayama : Director, Atsuo : Director, ...","Ryuuta Murayama\nDirector, Atsuo\nDirector, Sh..."
18487,Make My Day,Studio 5,"[Horror, Monsters, Original Work]","On a cold planet of ice and snow, mysterious c...","Yasuo Ootagaki : Original Creator, Makoto Kane...","Yasuo Ootagaki\nOriginal Creator, Makoto Kanem..."


In [ ]:
authors['Studio'] = authors['Studio'].apply(lambda x: [element.strip() for element in x])

<ipython-input-20-a1e49d1d6d51>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Studio'] = authors['Studio'].apply(lambda x: [element.strip() for element in x])


In [ ]:
authors['Studio'] = authors['Studio'].apply(lambda x: ' '.join(x))

<ipython-input-21-0037a2033d9f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Studio'] = authors['Studio'].apply(lambda x: ' '.join(x))


In [ ]:
authors['Studio'] = authors['Studio'].str.replace(' ', '')

<ipython-input-22-f5700a597c71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Studio'] = authors['Studio'].str.replace(' ', '')


In [ ]:
authors['Studio'] = authors['Studio'].apply(lambda x: [x])


<ipython-input-23-5f435c22544b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Studio'] = authors['Studio'].apply(lambda x: [x])


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...",'Tanjiro and his friends accompany the Hashira...,"Koyoharu Gotouge : Original Creator, Haruo Sot...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, Animal Trans...",'The final arc of Fruits Basket.',"Natsuki Takaya : Original Creator, Yoshihide I...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, Ancient China, Chinese Animation, Cu...",'The third season of Mo Dao Zu Shi.',"Mo Xiang Tong Xiu : Original Creator, Xiong Ke...","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","""The foundation of alchemy is based on the law...","Hiromu Arakawa : Original Creator, Yasuhiro Ir...","Alphonse Elric : Rie Kugimiya, Edward Elric : ..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, Dark Fantas...",'The battle to retake Wall Maria begins now! W...,"Hajime Isayama : Original Creator, Tetsurou Ar...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, School Club]",No synopsis yet - check back soon!,"Toshinao Aoki : Original Character Design, Kon...","Narrator : Haruka Ayase, Toshinao Aoki\nOrigin..."
18446,Ometeotl Hero,[Speed],"[Sci Fi, Anime Tamago, Shorts, Superpowers, Or...","In the year 22XX, due to the drastic decrease ...",Jun Awazu : Director,"Yuki, Jun Awazu\nDirector"
18482,Boris: Free,[foodunited.],[Abstract],The official music video for Free from the alb...,"Ryuuta Murayama : Director, Atsuo : Director, ...","Ryuuta Murayama\nDirector, Atsuo\nDirector, Sh..."
18487,Make My Day,[Studio5],"[Horror, Monsters, Original Work]","On a cold planet of ice and snow, mysterious c...","Yasuo Ootagaki : Original Creator, Makoto Kane...","Yasuo Ootagaki\nOriginal Creator, Makoto Kanem..."


In [ ]:
authors.head(1)['staff'].values

array(['Koyoharu Gotouge : Original Creator, Haruo Sotozaki : Director, Akira Matsushima : Character Design, Aimer : Song Performance'],
      dtype=object)

In [ ]:
authors['staff'] = authors['staff'].str.replace('\n', ' : ')
authors['Voice_actors'] = authors['Voice_actors'].str.replace('\n', ' : ')

<ipython-input-26-e969d2b0876d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['staff'] = authors['staff'].str.replace('\n', ' : ')
<ipython-input-26-e969d2b0876d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Voice_actors'] = authors['Voice_actors'].str.replace('\n', ' : ')


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...",'Tanjiro and his friends accompany the Hashira...,"Koyoharu Gotouge : Original Creator, Haruo Sot...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, Animal Trans...",'The final arc of Fruits Basket.',"Natsuki Takaya : Original Creator, Yoshihide I...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, Ancient China, Chinese Animation, Cu...",'The third season of Mo Dao Zu Shi.',"Mo Xiang Tong Xiu : Original Creator, Xiong Ke...","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","""The foundation of alchemy is based on the law...","Hiromu Arakawa : Original Creator, Yasuhiro Ir...","Alphonse Elric : Rie Kugimiya, Edward Elric : ..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, Dark Fantas...",'The battle to retake Wall Maria begins now! W...,"Hajime Isayama : Original Creator, Tetsurou Ar...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, School Club]",No synopsis yet - check back soon!,"Toshinao Aoki : Original Character Design, Kon...","Narrator : Haruka Ayase, Toshinao Aoki : Origi..."
18446,Ometeotl Hero,[Speed],"[Sci Fi, Anime Tamago, Shorts, Superpowers, Or...","In the year 22XX, due to the drastic decrease ...",Jun Awazu : Director,"Yuki, Jun Awazu : Director"
18482,Boris: Free,[foodunited.],[Abstract],The official music video for Free from the alb...,"Ryuuta Murayama : Director, Atsuo : Director, ...","Ryuuta Murayama : Director, Atsuo : Director, ..."
18487,Make My Day,[Studio5],"[Horror, Monsters, Original Work]","On a cold planet of ice and snow, mysterious c...","Yasuo Ootagaki : Original Creator, Makoto Kane...","Yasuo Ootagaki : Original Creator, Makoto Kane..."


In [ ]:
authors['staff'] = authors['staff'].apply(lambda x: [part.split(':')[0].strip() for part in x.split(',')])

<ipython-input-28-f40e6efd08d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['staff'] = authors['staff'].apply(lambda x: [part.split(':')[0].strip() for part in x.split(',')])


In [ ]:
authors['Voice_actors'] = authors['Voice_actors'].apply(lambda x: [part.split(':')[1].strip() if ':' in part else part.strip() for part in x.split(',')])

<ipython-input-29-d563d0e05c72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Voice_actors'] = authors['Voice_actors'].apply(lambda x: [part.split(':')[1].strip() if ':' in part else part.strip() for part in x.split(',')])


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...",'Tanjiro and his friends accompany the Hashira...,"[Koyoharu Gotouge, Haruo Sotozaki, Akira Matsu...","[Yoshitsugu Matsuoka, Akari Kitou, Natsuki Han..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, Animal Trans...",'The final arc of Fruits Basket.',"[Natsuki Takaya, Yoshihide Ibata, Taku Kishimo...","[Maaya Sakamoto, Yuuma Uchida, Yuuichi Nakamur..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, Ancient China, Chinese Animation, Cu...",'The third season of Mo Dao Zu Shi.',"[Mo Xiang Tong Xiu, Xiong Ke, Ma Chendi, Sun Y...","[Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guan..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","""The foundation of alchemy is based on the law...","[Hiromu Arakawa, Yasuhiro Irie, Akira Senju, H...","[Rie Kugimiya, Romi Park, Kenji Utsumi, Hideyu..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, Dark Fantas...",'The battle to retake Wall Maria begins now! W...,"[Hajime Isayama, Tetsurou Araki, Masashi Koizu...","[Marina Inoue, Yuuki Kaji, Yui Ishikawa, Armor..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, School Club]",No synopsis yet - check back soon!,"[Toshinao Aoki, Konousuke Uda, Mimori Yusa]","[Haruka Ayase, Original Character Design, Dire..."
18446,Ometeotl Hero,[Speed],"[Sci Fi, Anime Tamago, Shorts, Superpowers, Or...","In the year 22XX, due to the drastic decrease ...",[Jun Awazu],"[Yuki, Director]"
18482,Boris: Free,[foodunited.],[Abstract],The official music video for Free from the alb...,"[Ryuuta Murayama, Atsuo, Shoki Minami, Boris]","[Director, Director, Animation Director, Song ..."
18487,Make My Day,[Studio5],"[Horror, Monsters, Original Work]","On a cold planet of ice and snow, mysterious c...","[Yasuo Ootagaki, Makoto Kanemoto, Shouji Kawam...","[Original Creator, Director, Mechanical Design..."


In [ ]:
authors['Description'] = authors['Description'].str.replace("'", '')

<ipython-input-31-d3a03511b6f3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Description'] = authors['Description'].str.replace("'", '')


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...",Tanjiro and his friends accompany the Hashira ...,"[Koyoharu Gotouge, Haruo Sotozaki, Akira Matsu...","[Yoshitsugu Matsuoka, Akari Kitou, Natsuki Han..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, Animal Trans...",The final arc of Fruits Basket.,"[Natsuki Takaya, Yoshihide Ibata, Taku Kishimo...","[Maaya Sakamoto, Yuuma Uchida, Yuuichi Nakamur..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, Ancient China, Chinese Animation, Cu...",The third season of Mo Dao Zu Shi.,"[Mo Xiang Tong Xiu, Xiong Ke, Ma Chendi, Sun Y...","[Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guan..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","""The foundation of alchemy is based on the law...","[Hiromu Arakawa, Yasuhiro Irie, Akira Senju, H...","[Rie Kugimiya, Romi Park, Kenji Utsumi, Hideyu..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, Dark Fantas...",The battle to retake Wall Maria begins now! Wi...,"[Hajime Isayama, Tetsurou Araki, Masashi Koizu...","[Marina Inoue, Yuuki Kaji, Yui Ishikawa, Armor..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, School Club]",No synopsis yet - check back soon!,"[Toshinao Aoki, Konousuke Uda, Mimori Yusa]","[Haruka Ayase, Original Character Design, Dire..."
18446,Ometeotl Hero,[Speed],"[Sci Fi, Anime Tamago, Shorts, Superpowers, Or...","In the year 22XX, due to the drastic decrease ...",[Jun Awazu],"[Yuki, Director]"
18482,Boris: Free,[foodunited.],[Abstract],The official music video for Free from the alb...,"[Ryuuta Murayama, Atsuo, Shoki Minami, Boris]","[Director, Director, Animation Director, Song ..."
18487,Make My Day,[Studio5],"[Horror, Monsters, Original Work]","On a cold planet of ice and snow, mysterious c...","[Yasuo Ootagaki, Makoto Kanemoto, Shouji Kawam...","[Original Creator, Director, Mechanical Design..."


In [ ]:
authors['Description'] = authors['Description'].apply(lambda x:x.split())

<ipython-input-33-ba982588f397>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Description'] = authors['Description'].apply(lambda x:x.split())


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...","[Tanjiro, and, his, friends, accompany, the, H...","[Koyoharu Gotouge, Haruo Sotozaki, Akira Matsu...","[Yoshitsugu Matsuoka, Akari Kitou, Natsuki Han..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, Animal Trans...","[The, final, arc, of, Fruits, Basket.]","[Natsuki Takaya, Yoshihide Ibata, Taku Kishimo...","[Maaya Sakamoto, Yuuma Uchida, Yuuichi Nakamur..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, Ancient China, Chinese Animation, Cu...","[The, third, season, of, Mo, Dao, Zu, Shi.]","[Mo Xiang Tong Xiu, Xiong Ke, Ma Chendi, Sun Y...","[Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guan..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","[""The, foundation, of, alchemy, is, based, on,...","[Hiromu Arakawa, Yasuhiro Irie, Akira Senju, H...","[Rie Kugimiya, Romi Park, Kenji Utsumi, Hideyu..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, Dark Fantas...","[The, battle, to, retake, Wall, Maria, begins,...","[Hajime Isayama, Tetsurou Araki, Masashi Koizu...","[Marina Inoue, Yuuki Kaji, Yui Ishikawa, Armor..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, School Club]","[No, synopsis, yet, -, check, back, soon!]","[Toshinao Aoki, Konousuke Uda, Mimori Yusa]","[Haruka Ayase, Original Character Design, Dire..."
18446,Ometeotl Hero,[Speed],"[Sci Fi, Anime Tamago, Shorts, Superpowers, Or...","[In, the, year, 22XX,, due, to, the, drastic, ...",[Jun Awazu],"[Yuki, Director]"
18482,Boris: Free,[foodunited.],[Abstract],"[The, official, music, video, for, Free, from,...","[Ryuuta Murayama, Atsuo, Shoki Minami, Boris]","[Director, Director, Animation Director, Song ..."
18487,Make My Day,[Studio5],"[Horror, Monsters, Original Work]","[On, a, cold, planet, of, ice, and, snow,, mys...","[Yasuo Ootagaki, Makoto Kanemoto, Shouji Kawam...","[Original Creator, Director, Mechanical Design..."


In [ ]:
authors['Tags'] = authors['Tags'].apply(lambda x:[i.replace(" ", "") for i in x])
authors['staff'] = authors['staff'].apply(lambda x:[i.replace(" ", "") for i in x])

<ipython-input-35-0181a80debe5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['Tags'] = authors['Tags'].apply(lambda x:[i.replace(" ", "") for i in x])
<ipython-input-35-0181a80debe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['staff'] = authors['staff'].apply(lambda x:[i.replace(" ", "") for i in x])


In [ ]:
authors

,Name,Studio,Tags,Description,staff,Voice_actors
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...","[Tanjiro, and, his, friends, accompany, the, H...","[KoyoharuGotouge, HaruoSotozaki, AkiraMatsushi...","[Yoshitsugu Matsuoka, Akari Kitou, Natsuki Han..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, AnimalTransf...","[The, final, arc, of, Fruits, Basket.]","[NatsukiTakaya, YoshihideIbata, TakuKishimoto,...","[Maaya Sakamoto, Yuuma Uchida, Yuuichi Nakamur..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, AncientChina, ChineseAnimation, Cult...","[The, third, season, of, Mo, Dao, Zu, Shi.]","[MoXiangTongXiu, XiongKe, MaChendi, SunYujing,...","[Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guan..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","[""The, foundation, of, alchemy, is, based, on,...","[HiromuArakawa, YasuhiroIrie, AkiraSenju, Hiro...","[Rie Kugimiya, Romi Park, Kenji Utsumi, Hideyu..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, DarkFantasy...","[The, battle, to, retake, Wall, Maria, begins,...","[HajimeIsayama, TetsurouAraki, MasashiKoizuka,...","[Marina Inoue, Yuuki Kaji, Yui Ishikawa, Armor..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, SchoolClub]","[No, synopsis, yet, -, check, back, soon!]","[ToshinaoAoki, KonousukeUda, MimoriYusa]","[Haruka Ayase, Original Character Design, Dire..."
18446,Ometeotl Hero,[Speed],"[SciFi, AnimeTamago, Shorts, Superpowers, Orig...","[In, the, year, 22XX,, due, to, the, drastic, ...",[JunAwazu],"[Yuki, Director]"
18482,Boris: Free,[foodunited.],[Abstract],"[The, official, music, video, for, Free, from,...","[RyuutaMurayama, Atsuo, ShokiMinami, Boris]","[Director, Director, Animation Director, Song ..."
18487,Make My Day,[Studio5],"[Horror, Monsters, OriginalWork]","[On, a, cold, planet, of, ice, and, snow,, mys...","[YasuoOotagaki, MakotoKanemoto, ShoujiKawamori...","[Original Creator, Director, Mechanical Design..."


In [ ]:
authors['tags'] = authors['Description'] + authors['staff'] + authors['Studio'] + authors['Tags']

<ipython-input-37-a435679b1040>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors['tags'] = authors['Description'] + authors['staff'] + authors['Studio'] + authors['Tags']


In [ ]:
authors.drop('Voice_actors', axis=1, inplace=True)

<ipython-input-38-f82ef3692433>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors.drop('Voice_actors', axis=1, inplace=True)


In [ ]:
authors

,Name,Studio,Tags,Description,staff,tags
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,[ufotable],"[Action, Adventure, Fantasy, Shounen, Demons, ...","[Tanjiro, and, his, friends, accompany, the, H...","[KoyoharuGotouge, HaruoSotozaki, AkiraMatsushi...","[Tanjiro, and, his, friends, accompany, the, H..."
1,Fruits Basket the Final Season,[TMSEntertainment],"[Drama, Fantasy, Romance, Shoujo, AnimalTransf...","[The, final, arc, of, Fruits, Basket.]","[NatsukiTakaya, YoshihideIbata, TakuKishimoto,...","[The, final, arc, of, Fruits, Basket., Natsuki..."
2,Mo Dao Zu Shi 3,[B.CMAYPICTURES],"[Fantasy, AncientChina, ChineseAnimation, Cult...","[The, third, season, of, Mo, Dao, Zu, Shi.]","[MoXiangTongXiu, XiongKe, MaChendi, SunYujing,...","[The, third, season, of, Mo, Dao, Zu, Shi., Mo..."
3,Fullmetal Alchemist: Brotherhood,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...","[""The, foundation, of, alchemy, is, based, on,...","[HiromuArakawa, YasuhiroIrie, AkiraSenju, Hiro...","[""The, foundation, of, alchemy, is, based, on,..."
4,Attack on Titan 3rd Season: Part II,[WITStudio],"[Action, Fantasy, Horror, Shounen, DarkFantasy...","[The, battle, to, retake, Wall, Maria, begins,...","[HajimeIsayama, TetsurouAraki, MasashiKoizuka,...","[The, battle, to, retake, Wall, Maria, begins,..."
...,...,...,...,...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,[TYOAnimations],"[Drama, Sports, Basketball, SchoolClub]","[No, synopsis, yet, -, check, back, soon!]","[ToshinaoAoki, KonousukeUda, MimoriYusa]","[No, synopsis, yet, -, check, back, soon!, Tos..."
18446,Ometeotl Hero,[Speed],"[SciFi, AnimeTamago, Shorts, Superpowers, Orig...","[In, the, year, 22XX,, due, to, the, drastic, ...",[JunAwazu],"[In, the, year, 22XX,, due, to, the, drastic, ..."
18482,Boris: Free,[foodunited.],[Abstract],"[The, official, music, video, for, Free, from,...","[RyuutaMurayama, Atsuo, ShokiMinami, Boris]","[The, official, music, video, for, Free, from,..."
18487,Make My Day,[Studio5],"[Horror, Monsters, OriginalWork]","[On, a, cold, planet, of, ice, and, snow,, mys...","[YasuoOotagaki, MakotoKanemoto, ShoujiKawamori...","[On, a, cold, planet, of, ice, and, snow,, mys..."


In [ ]:
final_tags = authors[['Name', 'tags']]

In [ ]:
final_tags['tags'] = final_tags['tags'].apply(lambda x:" ".join(x))

<ipython-input-41-f05e987ecd28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tags['tags'] = final_tags['tags'].apply(lambda x:" ".join(x))


In [ ]:
final_tags

,Name,tags
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Tanjiro and his friends accompany the Hashira ...
1,Fruits Basket the Final Season,The final arc of Fruits Basket. NatsukiTakaya ...
2,Mo Dao Zu Shi 3,The third season of Mo Dao Zu Shi. MoXiangTong...
3,Fullmetal Alchemist: Brotherhood,"""The foundation of alchemy is based on the law..."
4,Attack on Titan 3rd Season: Part II,The battle to retake Wall Maria begins now! Wi...
...,...,...
18445,Onagawa Chuu Baske Bu: 5-nin no Natsu,No synopsis yet - check back soon! ToshinaoAok...
18446,Ometeotl Hero,"In the year 22XX, due to the drastic decrease ..."
18482,Boris: Free,The official music video for Free from the alb...
18487,Make My Day,"On a cold planet of ice and snow, mysterious c..."


In [ ]:
final_tags['tags'][0]

'Tanjiro and his friends accompany the Hashira Tengen Uzui to an entertainment district where Tengen’s female ninja agents were gathering information on a demon before they suddenly disappeared. In order to investigate, Tanjiro and the others disguise themselves as women to sneak in! KoyoharuGotouge HaruoSotozaki AkiraMatsushima Aimer ufotable Action Adventure Fantasy Shounen Demons Historical MartialArts Orphans Siblings Swordplay BasedonaManga ExplicitViolence'

In [ ]:
final_tags['tags'] = final_tags['tags'].apply(lambda x:x.lower())

<ipython-input-44-3aeef4be4008>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tags['tags'] = final_tags['tags'].apply(lambda x:x.lower())


In [ ]:
!pip install fuzzywuzzy

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

# Fuzzy Wuzzy

In [ ]:
matches = {}

In [ ]:
for title in anime['title']:
    match = process.extractOne(title, final_tags['Name'])

    if match[1] >= 90:
        matches[title] = match[0]
anime['matched_anime_name'] = anime['title'].map(matches)

ModuleNotFoundError: ignored

In [ ]:
merged_df = pd.merge(anime, final_tags, left_on='title', right_on='Name', how='left')
merged_df.drop(['Name'], axis=1, inplace=True)
merged_df.rename(columns={'title': 'anime'}, inplace=True)

In [ ]:
merged_df

,uid,anime,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,tags
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,"""after losing to aoba johsai at the inter-high..."
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,NaN
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,"""young riko has spent her entire childhood dre..."
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,"""the foundation of alchemy is based on the law..."
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19315,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13.0,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,NaN
19316,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52.0,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,NaN
19317,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19.0,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,NaN
19318,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,NaN


# Tfidf

In [ ]:
cv = CountVectorizer(stop_words='english')

In [ ]:
vector = cv.fit_transform(final_tags['tags']).toarray()
vector.shape

(9587, 42286)

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

array([[1.        , 0.08122955, 0.03370999, ..., 0.        , 0.02956562,
        0.03143473],
       [0.08122955, 1.        , 0.04016097, ..., 0.        , 0.        ,
        0.03745029],
       [0.03370999, 0.04016097, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.02956562, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.03143473, 0.03745029, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
final_tags[final_tags['Name'] == 'Haikyuu!! Second Season'].index[0]

11

In [ ]:
def recommend(anime):
    index = final_tags[final_tags['Name'] == anime].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:20]:
        print(final_tags.iloc[i[0]].Name)

In [ ]:
recommend('Haikyuu!! Second Season')

Haikyuu!! Movie 2: Shousha to Haisha
Haikyuu!! Movie 1: Owari to Hajimari
Haikyuu!! Movie 3: Talent and Sense
Haikyuu!! Karasuno High School vs Shiratorizawa Academy
Haikyuu!! Riku vs Kuu
Haikyuu!! To the Top
Haikyuu!!
Haikyuu!! To the Top: Part II
Haikyuu!! Tokushuu! Harukou Volley ni Kaketa Seishun
Haikyuu!! vs. "Akaten"
Love All Play
Attacker You!
Haikyuu!! Movie 4: Battle of Concepts
The God of High School
Yowamushi Pedal The Movie
Attack No. 1
Ashita E Attack!
Yowamushi Pedal: Glory Line
Slam Dunk Movie 1


# NLP {Multi Label}

In [ ]:
for_nlp = final_tags = authors[['Description', 'Tags']]

In [ ]:
for_nlp

,Description,Tags
0,"[Tanjiro, and, his, friends, accompany, the, H...","[Action, Adventure, Fantasy, Shounen, Demons, ..."
1,"[The, final, arc, of, Fruits, Basket.]","[Drama, Fantasy, Romance, Shoujo, AnimalTransf..."
2,"[The, third, season, of, Mo, Dao, Zu, Shi.]","[Fantasy, AncientChina, ChineseAnimation, Cult..."
3,"[""The, foundation, of, alchemy, is, based, on,...","[Action, Adventure, Drama, Fantasy, Mystery, S..."
4,"[The, battle, to, retake, Wall, Maria, begins,...","[Action, Fantasy, Horror, Shounen, DarkFantasy..."
...,...,...
18445,"[No, synopsis, yet, -, check, back, soon!]","[Drama, Sports, Basketball, SchoolClub]"
18446,"[In, the, year, 22XX,, due, to, the, drastic, ...","[SciFi, AnimeTamago, Shorts, Superpowers, Orig..."
18482,"[The, official, music, video, for, Free, from,...",[Abstract]
18487,"[On, a, cold, planet, of, ice, and, snow,, mys...","[Horror, Monsters, OriginalWork]"


In [ ]:
for_nlp['Description'] = for_nlp['Description'].apply(lambda x: ' '.join(map(str, x)))

<ipython-input-59-6efbb76508b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_nlp['Description'] = for_nlp['Description'].apply(lambda x: ' '.join(map(str, x)))


In [ ]:
for_nlp

,Description,Tags
0,Tanjiro and his friends accompany the Hashira ...,"[Action, Adventure, Fantasy, Shounen, Demons, ..."
1,The final arc of Fruits Basket.,"[Drama, Fantasy, Romance, Shoujo, AnimalTransf..."
2,The third season of Mo Dao Zu Shi.,"[Fantasy, AncientChina, ChineseAnimation, Cult..."
3,"""The foundation of alchemy is based on the law...","[Action, Adventure, Drama, Fantasy, Mystery, S..."
4,The battle to retake Wall Maria begins now! Wi...,"[Action, Fantasy, Horror, Shounen, DarkFantasy..."
...,...,...
18445,No synopsis yet - check back soon!,"[Drama, Sports, Basketball, SchoolClub]"
18446,"In the year 22XX, due to the drastic decrease ...","[SciFi, AnimeTamago, Shorts, Superpowers, Orig..."
18482,The official music video for Free from the alb...,[Abstract]
18487,"On a cold planet of ice and snow, mysterious c...","[Horror, Monsters, OriginalWork]"


In [ ]:
y = for_nlp['Tags']
y

0        [Action, Adventure, Fantasy, Shounen, Demons, ...
1        [Drama, Fantasy, Romance, Shoujo, AnimalTransf...
2        [Fantasy, AncientChina, ChineseAnimation, Cult...
3        [Action, Adventure, Drama, Fantasy, Mystery, S...
4        [Action, Fantasy, Horror, Shounen, DarkFantasy...
                               ...                        
18445              [Drama, Sports, Basketball, SchoolClub]
18446    [SciFi, AnimeTamago, Shorts, Superpowers, Orig...
18482                                           [Abstract]
18487                     [Horror, Monsters, OriginalWork]
18488    [Comedy, SliceofLife, Ghosts, Iyashikei, Non-H...
Name: Tags, Length: 9587, dtype: object

In [ ]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(for_nlp['Tags'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
multilabel.classes_

array(['15thCentury', '16thCentury', '17thCentury', '18thCentury',
       '19thCentury', 'Abstract', 'Acting', 'Action', 'AdultCouples',
       'AdultIndustry', 'Adventure', 'Africa', 'Afterlife', 'AgeGap',
       'AgeTransformation', 'Aging', 'Agriculture', 'Alcohol',
       'AliceinWonderland', 'Aliens', 'All-BoysSchool', 'All-GirlsSchool',
       'America', 'Amnesia', 'AmusementPark', 'AncientChina', 'Androids',
       'Androphobia', 'Angels', 'AnimalAbuse,', 'AnimalCharacteristics',
       'AnimalProtagonists', 'AnimalTransformation', 'AnimeBancho',
       'AnimeIndustry', 'AnimeTamago', 'Animeism', 'AnimenoChikara',
       'AnimenoMe', 'Anthropomorphic', 'ApartmentLife', 'Apocalypse',
       'Apprenticeship', 'Archery', 'ArrangedMarriage', 'Art',
       'ArtSchool', 'Assassins', 'Astronauts', 'Astronomy', 'Australia',
       'Autobiographies', 'Aviation', 'BDSM', 'BL', 'Badminton', 'Baking',
       'BalletDancing', 'Bar', 'Baseball', 'Basedona4-KomaManga',
       'BasedonaCardGame

In [ ]:
pd.DataFrame(y, columns=multilabel.classes_)

,15thCentury,16thCentury,17thCentury,18thCentury,19thCentury,Abstract,Acting,Action,AdultCouples,AdultIndustry,...,Wuxia,Xianxia,Yakuza,Yaoi,YaoiHandSyndrome,Youkai,Yuri,Zombies,Zoo,noitaminA
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9584,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,3), stop_words='english')
X = tfidf.fit_transform(for_nlp['Description'])
X.shape, y.shape

((9587, 10000), (9587, 547))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()

In [ ]:
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
  return jaccard.mean()*100


def print_score(y_pred, clf):
  print("Clf: ", clf.__class__.__name__)
  print('Jacard score: {}'.format(j_score(y_test, y_pred)))
  print('----')

In [ ]:
for classifier in [LinearSVC(C=1.5, penalty='l1', dual=False, max_iter=20000)]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Clf:  LinearSVC
Jacard score: 22.05611202304376
----


In [ ]:
for classifier in [sgd, lr, svc]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print_score(y_pred, classifier)

/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 112 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 169 is present in all training examples.
  warnings.warn(


Clf:  SGDClassifier
Jacard score: 19.026947037318877
----


/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 112 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 169 is present in all training examples.
  warnings.warn(


Clf:  LogisticRegression
Jacard score: 8.287600003693182
----


/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 112 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 169 is present in all training examples.
  warnings.warn(


Clf:  LinearSVC
Jacard score: 20.236476903576033
----


In [ ]:
x = [ 'Determined to be like the volleyball championships star player nicknamed the small giant, Shoyo joins his schools volleyball club.']
xt = tfidf.transform(x)
predicted_labels = clf.predict(xt)

In [ ]:
label_names = clf.classes_

In [ ]:
label_names = multilabel.classes_

label_names_list = [label_names[i] for i in range(len(label_names))]

print(label_names_list)

['15thCentury', '16thCentury', '17thCentury', '18thCentury', '19thCentury', 'Abstract', 'Acting', 'Action', 'AdultCouples', 'AdultIndustry', 'Adventure', 'Africa', 'Afterlife', 'AgeGap', 'AgeTransformation', 'Aging', 'Agriculture', 'Alcohol', 'AliceinWonderland', 'Aliens', 'All-BoysSchool', 'All-GirlsSchool', 'America', 'Amnesia', 'AmusementPark', 'AncientChina', 'Androids', 'Androphobia', 'Angels', 'AnimalAbuse,', 'AnimalCharacteristics', 'AnimalProtagonists', 'AnimalTransformation', 'AnimeBancho', 'AnimeIndustry', 'AnimeTamago', 'Animeism', 'AnimenoChikara', 'AnimenoMe', 'Anthropomorphic', 'ApartmentLife', 'Apocalypse', 'Apprenticeship', 'Archery', 'ArrangedMarriage', 'Art', 'ArtSchool', 'Assassins', 'Astronauts', 'Astronomy', 'Australia', 'Autobiographies', 'Aviation', 'BDSM', 'BL', 'Badminton', 'Baking', 'BalletDancing', 'Bar', 'Baseball', 'Basedona4-KomaManga', 'BasedonaCardGame', 'BasedonaCartoon', 'BasedonaComicBook', 'BasedonaDoujinshi', 'BasedonaFairyTale', 'BasedonaLightNovel

In [ ]:
predicted_labels = np.where(predicted_labels == 1)[1]

predicted_labels = [label_names_list[i] for i in predicted_labels]

print(predicted_labels)

['BasedonaManga', 'SchoolClub', 'SchoolLife', 'Sports', 'Volleyball']


In [ ]:
anime

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13.0,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52.0,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19.0,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...


In [ ]:
users

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99
...,...,...,...,...,...
81722,lovelessxd,Female,"Aug 6, 1992","['853', '5114']",https://myanimelist.net/profile/lovelessxd
81723,Shattered_Angel,Female,"Sep 6, 1994","['150', '27', '1520', '121', '31452', '32995',...",https://myanimelist.net/profile/Shattered_Angel
81724,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus
81725,camco,Female,Sep 23,"['199', '4224', '7054', '13601', '14713']",https://myanimelist.net/profile/camco


In [ ]:
ratings

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149
...,...,...,...,...,...,...,...
192107,240067,Unicorn819,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '5', 'Animation': '1...",https://myanimelist.net/reviews.php?id=240067
192108,285777,ShizzoSVH,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=285777
192109,286904,AlluMan96,1281,\n \n \n \n ...,3,"{'Overall': '3', 'Story': '3', 'Animation': '1...",https://myanimelist.net/reviews.php?id=286904
192110,287903,AgentK300,1281,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '3', 'Animation': '...",https://myanimelist.net/reviews.php?id=287903


In [ ]:
users['uid'] = users.index

In [ ]:
users

,profile,gender,birthday,favorites_anime,link,uid
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche,0
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans,1
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn,2
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00,3
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99,4
...,...,...,...,...,...,...
81722,lovelessxd,Female,"Aug 6, 1992","['853', '5114']",https://myanimelist.net/profile/lovelessxd,81722
81723,Shattered_Angel,Female,"Sep 6, 1994","['150', '27', '1520', '121', '31452', '32995',...",https://myanimelist.net/profile/Shattered_Angel,81723
81724,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus,81724
81725,camco,Female,Sep 23,"['199', '4224', '7054', '13601', '14713']",https://myanimelist.net/profile/camco,81725


In [ ]:
ratings['scores'][0]

"{'Overall': '8', 'Story': '8', 'Animation': '8', 'Sound': '10', 'Character': '9', 'Enjoyment': '8'}"

In [ ]:
normalized_ratings = json_normalize(ratings['scores'])

In [ ]:
normalized_ratings.info

<bound method DataFrame.info of Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[192112 rows x 0 columns]>

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192112 entries, 0 to 192111
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   uid        192112 non-null  int64 
 1   profile    192112 non-null  object
 2   anime_uid  192112 non-null  int64 
 3   text       192112 non-null  object
 4   score      192112 non-null  int64 
 5   scores     192112 non-null  object
 6   link       192112 non-null  object
dtypes: int64(3), object(4)
memory usage: 10.3+ MB


In [ ]:
ratings['scores'] = ratings['scores'].apply(literal_eval)

In [ ]:
if all(isinstance(score, dict) for score in ratings['scores']):
    normalized_scores = json_normalize(ratings['scores'])
    ratings = pd.concat([ratings, normalized_scores], axis=1)
    print(ratings)

           uid          profile  anime_uid  \
0       255938   DesolatePsyche      34096   
1       259117        baekbeans      34599   
2       253664             skrn      28891   
3         8254     edgewalker00       2904   
4       291149  aManOfCulture99       4181   
...        ...              ...        ...   
192107  240067       Unicorn819       1281   
192108  285777        ShizzoSVH       1281   
192109  286904        AlluMan96       1281   
192110  287903        AgentK300       1281   
192111  299323       Scarlet012       1281   

                                                     text  score  \
0       \n           \n         \n           \n       ...      8   
1       \n           \n         \n           \n       ...     10   
2       \n           \n         \n           \n       ...      7   
3       \n           \n         \n           \n       ...      9   
4       \n           \n         \n           \n       ...     10   
...                                    

In [ ]:
ratings

,uid,profile,anime_uid,text,score,scores,link,Overall,Story,Animation,Sound,Character,Enjoyment
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938,8,8,8,10,9,8
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117,10,10,10,10,10,10
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664,7,7,9,8,8,8
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254,9,9,9,10,10,9
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149,10,10,8,9,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192107,240067,Unicorn819,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '5', 'Animation': '1...",https://myanimelist.net/reviews.php?id=240067,9,5,10,10,9,9
192108,285777,ShizzoSVH,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=285777,9,7,9,9,9,10
192109,286904,AlluMan96,1281,\n \n \n \n ...,3,"{'Overall': '3', 'Story': '3', 'Animation': '1...",https://myanimelist.net/reviews.php?id=286904,3,3,1,2,2,5
192110,287903,AgentK300,1281,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '3', 'Animation': '...",https://myanimelist.net/reviews.php?id=287903,10,3,3,3,3,10


In [ ]:
anime = anime.rename(columns={'uid': 'anime_uid'})

In [ ]:
ratings_with_name = ratings.merge(anime, on='anime_uid')

In [ ]:
ratings_with_name

,uid,profile,anime_uid,text,score_x,scores,link_x,Overall,Story,Animation,...,synopsis,genre,aired,episodes,members,popularity,ranked,score_y,img_url,link_y
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938,8,8,8,...,After joining the resistance against the bakuf...,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,139309,800,15.0,8.94,https://cdn.myanimelist.net/images/anime/3/835...,https://myanimelist.net/anime/34096/Gintama
1,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938,8,8,8,...,After joining the resistance against the bakuf...,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,139309,800,15.0,8.94,https://cdn.myanimelist.net/images/anime/3/835...,https://myanimelist.net/anime/34096/Gintama
2,249891,claudinou,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '7', 'Animation': '8...",https://myanimelist.net/reviews.php?id=249891,8,7,8,...,After joining the resistance against the bakuf...,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,139309,800,15.0,8.94,https://cdn.myanimelist.net/images/anime/3/835...,https://myanimelist.net/anime/34096/Gintama
3,249891,claudinou,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '7', 'Animation': '8...",https://myanimelist.net/reviews.php?id=249891,8,7,8,...,After joining the resistance against the bakuf...,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,139309,800,15.0,8.94,https://cdn.myanimelist.net/images/anime/3/835...,https://myanimelist.net/anime/34096/Gintama
4,313501,PeterFromRussia,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=313501,8,8,8,...,After joining the resistance against the bakuf...,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,139309,800,15.0,8.94,https://cdn.myanimelist.net/images/anime/3/835...,https://myanimelist.net/anime/34096/Gintama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317474,233376,Kuromizue,9751,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '8...",https://myanimelist.net/reviews.php?id=233376,9,9,8,...,After fending off the threat of a Neuroi invas...,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,32054,2464,1287.0,7.65,https://cdn.myanimelist.net/images/anime/1133/...,https://myanimelist.net/anime/9751/Strike_Witc...
317475,77077,ryanxwonbin,9751,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '7', 'Animation': '8...",https://myanimelist.net/reviews.php?id=77077,8,7,8,...,After fending off the threat of a Neuroi invas...,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,32054,2464,1287.0,7.65,https://cdn.myanimelist.net/images/anime/1133/...,https://myanimelist.net/anime/9751/Strike_Witc...
317476,231781,AobaSuzukaze,9751,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '0', 'Animation': '...",https://myanimelist.net/reviews.php?id=231781,10,0,0,...,After fending off the threat of a Neuroi invas...,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,32054,2464,1287.0,7.65,https://cdn.myanimelist.net/images/anime/1133/...,https://myanimelist.net/anime/9751/Strike_Witc...
317477,78776,7jaws7,9751,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=78776,9,9,9,...,After fending off the threat of a Neuroi invas...,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,32054,2464,1287.0,7.65,https://cdn.myanimelist.net/images/anime/1133/...,https://myanimelist.net/anime/9751/Strike_Witc...


In [ ]:
ratings_with_name = ratings_with_name[['uid', 'anime_uid', 'score_x']]

In [ ]:
ratings_with_name.duplicated().sum()

186960

In [ ]:
ratings_with_name.drop_duplicates(inplace=True)

<ipython-input-132-8a09aed61a32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_with_name.drop_duplicates(inplace=True)


In [ ]:
ratings_with_name

,uid,anime_uid,score_x
0,255938,34096,8
2,249891,34096,8
4,313501,34096,8
6,317041,34096,8
8,245369,34096,7
...,...,...,...
317474,233376,9751,9
317475,77077,9751,8
317476,231781,9751,10
317477,78776,9751,9


In [ ]:
x = ratings_with_name.groupby('uid').count()['score_x'] > 10
padhe_likhe_users = x[x].index

In [ ]:
ratings_with_name[ratings_with_name['uid'].isin(padhe_likhe_users)]

,uid,anime_uid,score_x


# Another Method

In [ ]:
users

,profile,gender,birthday,favorites_anime,link,uid
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche,0
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans,1
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn,2
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00,3
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99,4
...,...,...,...,...,...,...
81722,lovelessxd,Female,"Aug 6, 1992","['853', '5114']",https://myanimelist.net/profile/lovelessxd,81722
81723,Shattered_Angel,Female,"Sep 6, 1994","['150', '27', '1520', '121', '31452', '32995',...",https://myanimelist.net/profile/Shattered_Angel,81723
81724,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus,81724
81725,camco,Female,Sep 23,"['199', '4224', '7054', '13601', '14713']",https://myanimelist.net/profile/camco,81725


In [ ]:
users['favorites_anime'][0]

"['33352', '25013', '5530', '33674', '1482', '269', '18245', '2904', '27899', '17074', '12291', '226', '28851', '8525', '6594', '4981', '1698', '457', '235', '34618']"

In [ ]:
users['favorites_anime'] = users['favorites_anime'].apply(literal_eval)

In [ ]:
users

,profile,gender,birthday,favorites_anime,link,uid
0,DesolatePsyche,Male,"Oct 2, 1994","[33352, 25013, 5530, 33674, 1482, 269, 18245, ...",https://myanimelist.net/profile/DesolatePsyche,0
1,baekbeans,Female,"Nov 10, 2000","[11061, 31964, 853, 20583, 918, 9253, 34599, 3...",https://myanimelist.net/profile/baekbeans,1
2,skrn,NaN,NaN,"[918, 2904, 11741, 17074, 23273, 32281, 9989, ...",https://myanimelist.net/profile/skrn,2
3,edgewalker00,Male,Sep 5,"[5680, 849, 2904, 3588, 37349]",https://myanimelist.net/profile/edgewalker00,3
4,aManOfCulture99,Male,"Oct 30, 1999","[4181, 7791, 9617, 5680, 2167, 4382, 849, 235,...",https://myanimelist.net/profile/aManOfCulture99,4
...,...,...,...,...,...,...
81722,lovelessxd,Female,"Aug 6, 1992","[853, 5114]",https://myanimelist.net/profile/lovelessxd,81722
81723,Shattered_Angel,Female,"Sep 6, 1994","[150, 27, 1520, 121, 31452, 32995, 877, 14713,...",https://myanimelist.net/profile/Shattered_Angel,81723
81724,FluffyWalrus,Male,NaN,"[121, 43, 237, 202, 205]",https://myanimelist.net/profile/FluffyWalrus,81724
81725,camco,Female,Sep 23,"[199, 4224, 7054, 13601, 14713]",https://myanimelist.net/profile/camco,81725


In [ ]:
users_exploded = users.explode('favorites_anime')

In [ ]:
users_exploded

,profile,gender,birthday,favorites_anime,link,uid
0,DesolatePsyche,Male,"Oct 2, 1994",33352,https://myanimelist.net/profile/DesolatePsyche,0
0,DesolatePsyche,Male,"Oct 2, 1994",25013,https://myanimelist.net/profile/DesolatePsyche,0
0,DesolatePsyche,Male,"Oct 2, 1994",5530,https://myanimelist.net/profile/DesolatePsyche,0
0,DesolatePsyche,Male,"Oct 2, 1994",33674,https://myanimelist.net/profile/DesolatePsyche,0
0,DesolatePsyche,Male,"Oct 2, 1994",1482,https://myanimelist.net/profile/DesolatePsyche,0
...,...,...,...,...,...,...
81726,nyja-chan,Female,Nov 13,2251,https://myanimelist.net/profile/nyja-chan,81726
81726,nyja-chan,Female,Nov 13,37779,https://myanimelist.net/profile/nyja-chan,81726
81726,nyja-chan,Female,Nov 13,13125,https://myanimelist.net/profile/nyja-chan,81726
81726,nyja-chan,Female,Nov 13,35180,https://myanimelist.net/profile/nyja-chan,81726


In [ ]:
final_users = users_exploded[['uid', 'favorites_anime']]

In [ ]:
final_users

,uid,favorites_anime
0,0,33352
0,0,25013
0,0,5530
0,0,33674
0,0,1482
...,...,...
81726,81726,2251
81726,81726,37779
81726,81726,13125
81726,81726,35180


In [ ]:
final_users['favorites_anime'] = final_users['favorites_anime'].fillna('')

tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(', '))
anime_matrix = tfidf.fit_transform(final_users['favorites_anime'])

num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
final_users['cluster'] = kmeans.fit_predict(anime_matrix)

user_id_to_recommend = 1
user_cluster = final_users[final_users['uid'] == user_id_to_recommend]['cluster'].values[0]

similar_users = final_users[final_users['cluster'] == user_cluster]

similar_users = similar_users[similar_users['uid'] != user_id_to_recommend]

similar_anime_matrix = tfidf.transform(similar_users['favorites_anime'])

kmeans_anime = KMeans(n_clusters=5, random_state=42)
similar_users['anime_cluster'] = kmeans_anime.fit_predict(similar_anime_matrix)

user_anime_cluster = final_users[final_users['uid'] == user_id_to_recommend]['cluster'].values[0]
recommended_anime = similar_users[similar_users['anime_cluster'] == user_anime_cluster]['favorites_anime']

user_fav_anime = final_users[final_users['uid'] == user_id_to_recommend]['favorites_anime'].values[0].split(', ')
recommended_anime = recommended_anime[~recommended_anime.isin(user_fav_anime)]

print("Number of Recommendations:", len(recommended_anime))
print("Recommended Anime:", recommended_anime.values)

<ipython-input-21-5d5f06adfdd2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_users['favorites_anime'] = final_users['favorites_anime'].fillna('')
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-21-5d5f06adfdd2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Number of Recommendations: 354396
Recommended Anime: ['33352' '25013' '5530' ... '13125' '35180' '10721']


In [ ]:
final_users['favorites_anime'] = final_users['favorites_anime'].fillna('')

# Create a TF-IDF matrix for anime preferences
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(', '))
anime_matrix = tfidf.fit_transform(final_users['favorites_anime'])

# Apply KMeans clustering on the TF-IDF matrix
num_clusters = 5  # You need to choose an appropriate number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
final_users['cluster'] = kmeans.fit_predict(anime_matrix)

# Given a user, find the cluster and recommend similar users' favorite anime
user_id_to_recommend = 1  # Replace this with the user you want to recommend anime for
user_cluster = final_users[final_users['uid'] == user_id_to_recommend]['cluster'].values[0]

# Extract users in the same cluster
similar_users = final_users[final_users['cluster'] == user_cluster]

# Remove the target user from similar users (optional)
similar_users = similar_users[similar_users['uid'] != user_id_to_recommend]

# Extract their favorite anime
similar_anime_matrix = tfidf.transform(similar_users['favorites_anime'])

# Apply KMeans on similar users' favorite anime
kmeans_anime = KMeans(n_clusters=5, random_state=42)  # Adjust the number of clusters
similar_users['anime_cluster'] = kmeans_anime.fit_predict(similar_anime_matrix)

# Recommend anime from the nearest cluster
user_anime_cluster = final_users[final_users['uid'] == user_id_to_recommend]['cluster'].values[0]
recommended_anime = similar_users[similar_users['anime_cluster'] == user_anime_cluster]['favorites_anime']

# Remove anime already in the user's favorite list
user_fav_anime = final_users[final_users['uid'] == user_id_to_recommend]['favorites_anime'].values[0].split(', ')
recommended_anime = recommended_anime[~recommended_anime.isin(user_fav_anime)]

# Get only the top 3 recommendations
top3_recommendations = recommended_anime.head(3)

print("Number of Recommendations:", len(top3_recommendations))
print("Top 3 Recommended Anime:", top3_recommendations.values)


<ipython-input-22-6434269dcab4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_users['favorites_anime'] = final_users['favorites_anime'].fillna('')
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-22-6434269dcab4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Number of Recommendations: 3
Top 3 Recommended Anime: ['33352' '25013' '5530']


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

# Assuming 'uid' and 'favorites_anime' are both integer columns
final_users.loc[:, 'favorites_anime'] = final_users['favorites_anime'].fillna('')  # Replace NaN with an empty string

# Remove duplicate entries
final_users_unique = final_users.drop_duplicates(subset=['uid', 'favorites_anime'])

# Reset the index to avoid duplicate index values
final_users_unique = final_users_unique.reset_index(drop=True)

# Prepare user-anime interaction matrix
interaction_matrix = final_users_unique.pivot_table(index='uid', columns='favorites_anime', aggfunc='size', fill_value=0)

# Check for and handle duplicate columns
if interaction_matrix.columns.duplicated().any():
    interaction_matrix = interaction_matrix.groupby(level=0, axis=1).sum()

# Apply KMeans clustering on the user-anime interaction matrix
num_clusters = 5  # You need to choose an appropriate number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
final_users_unique['cluster'] = kmeans.fit_predict(interaction_matrix)

# Given a user, find the cluster and recommend similar users' favorite anime
user_id_to_recommend = 1  # Replace this with the user you want to recommend anime for
user_cluster = final_users_unique[final_users_unique['uid'] == user_id_to_recommend]['cluster'].values[0]

# Extract users in the same cluster
similar_users = final_users_unique[final_users_unique['cluster'] == user_cluster]

# Remove the target user from similar users (optional)
similar_users = similar_users[similar_users['uid'] != user_id_to_recommend]

# Extract their favorite anime
similar_anime_counts = similar_users.pivot_table(index='uid', columns='favorites_anime', aggfunc='size', fill_value=0)

# Check for and handle duplicate columns in similar anime counts
if similar_anime_counts.columns.duplicated().any():
    similar_anime_counts = similar_anime_counts.groupby(level=0, axis=1).sum()

# Apply KMeans on similar users' favorite anime
kmeans_anime = KMeans(n_clusters=5, random_state=42)  # Adjust the number of clusters
similar_anime_counts['anime_cluster'] = kmeans_anime.fit_predict(similar_anime_counts)

# Recommend anime from the nearest cluster
user_anime_cluster = final_users_unique[final_users_unique['uid'] == user_id_to_recommend]['cluster'].values[0]
recommended_anime_cluster = similar_anime_counts[similar_anime_counts['anime_cluster'] == user_anime_cluster].index

# Remove anime already in the user's favorite list
user_fav_anime = final_users_unique[final_users_unique['uid'] == user_id_to_recommend]['favorites_anime'].values[0]
recommended_anime = [anime for anime in recommended_anime_cluster if anime not in user_fav_anime]

# Get only the top 3 recommendations
top3_recommendations = recommended_anime[:3]

print("Number of Recommendations:", len(top3_recommendations))
print("Top 3 Recommended Anime:", top3_recommendations)


<ipython-input-23-65d5632faa5c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_users.loc[:, 'favorites_anime'] = final_users['favorites_anime'].fillna('')  # Replace NaN with an empty string
